# Downloading data to project

In [0]:
import warnings
warnings.filterwarnings("ignore")

import urllib3
link = 'https://raw.githubusercontent.com/jerryjerryj/Jobs2/master/Data/Vacancies'
wiki_link = 'https://raw.githubusercontent.com/jerryjerryj/Jobs2/master/Data/Wiki.txt'
pool_manager = urllib3.PoolManager()

response = pool_manager.request('GET',link+'/HH.txt')
raw_hh = response.data.decode('utf-8').splitlines()

response = pool_manager.request('GET',link+'/SJ.txt')
raw_sj = response.data.decode('utf-8').splitlines()

# Data cleansing and tokenization (with lemmatization)

In [0]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem

def TokenizeSentencesLemmatized(rawSentences):
    print('LEMMATIZED total = ' +str(rawSentences.__len__()))
    sentences = []
    m = Mystem()
    index = 0
    for c in rawSentences:
        tokenized_sents = m.lemmatize(c)
        cleaned_set = []
        for tokenized in tokenized_sents:
            if tokenized == "":
                break
            tokenized = tokenized.lower()
            if tokenized in stopwords.words('russian'):
                continue

            token = tokenized[0]
            if (token >= 'а' and token <= 'я'):
                cleaned_set.append(tokenized)
            elif ((token >= 'а' and token <= 'я') or (token >= 'a' and token <= 'z')):
                cleaned_set.append(tokenized)

        if cleaned_set.__len__()>0:
            sentences.append(cleaned_set)
        if index%1000 == 0:
          print(index)
        index+=1
    return sentences


# LDA

In [0]:
tokenized = TokenizeSentencesLemmatized(raw_hh) +TokenizeSentencesLemmatized(raw_sj)


In [29]:
import gensim
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

dictionary = Dictionary(tokenized)
corpus = [dictionary.doc2bow(t) for t in tokenized]
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=12, update_every=1, chunksize=10000, passes=1)

lda.print_topics(10)

[(7,
  '0.020*"работа" + 0.019*"опыт" + 0.017*"знание" + 0.013*"разработка" + 0.011*"система" + 0.010*"требование" + 0.007*"решение" + 0.007*"компания" + 0.007*"проект" + 0.006*"умение"'),
 (4,
  '0.019*"знание" + 0.019*"работа" + 0.013*"опыт" + 0.012*"технический" + 0.010*"система" + 0.009*"разработка" + 0.008*"требование" + 0.007*"проект" + 0.007*"умение" + 0.006*"компания"'),
 (10,
  '0.022*"работа" + 0.016*"опыт" + 0.012*"знание" + 0.009*"компания" + 0.008*"разработка" + 0.007*"сайт" + 0.007*"требование" + 0.006*"проект" + 0.006*"информация" + 0.006*"система"'),
 (9,
  '0.036*"опыт" + 0.030*"работа" + 0.019*"знание" + 0.014*"система" + 0.010*"разработка" + 0.008*"данные" + 0.008*"требование" + 0.007*"администрирование" + 0.006*"компания" + 0.005*"задача"'),
 (0,
  '0.020*"работа" + 0.016*"технический" + 0.013*"разработка" + 0.011*"опыт" + 0.011*"информационный" + 0.010*"безопасность" + 0.010*"система" + 0.009*"знание" + 0.008*"документация" + 0.007*"тестирование"'),
 (11,
  '0.028*

# lda +tfidf

In [30]:
from keras.preprocessing.text import Tokenizer
def get_texts_to_matrix(texts, max_features=0):
    tokenizer = Tokenizer(split=" ", lower=True)
    if max_features != 0:
        tokenizer = Tokenizer(split=" ", lower=True, num_words=max_features,char_level='True')

    tokenizer.fit_on_texts(texts)
    matrix_tfidf = tokenizer.texts_to_matrix(texts=texts, mode='tfidf')
    print('Количество текстов:', matrix_tfidf.shape[0])
    print('Количество токенов:', matrix_tfidf.shape[1])
    return matrix_tfidf, tokenizer.word_index

Using TensorFlow backend.


In [50]:
tfidf, dictionary = get_texts_to_matrix(tokenized)
treshold = 4

new_sentences=[]
for i in range(0,tokenized.__len__()):
  sentence = tokenized[i]
  s_tfidf = tfidf[i]
  new_sentence = []
  for word in sentence:
    word_id = dictionary[word]
    word_tfidf = s_tfidf[word_id]
    if word_tfidf>treshold:
        new_sentence.append(word)
  new_sentences.append(new_sentence)
print(new_sentences[:10])

Количество текстов: 1563
Количество токенов: 9980
[['бэкенд', 'net', 'покрытие', 'unit', 'тесто', 'постановка', 'frontend', 'архитектурный', 'постановка', 'спринт', 'демо', 'code', 'review', 'net', 'webapi', 'linq2sql', 'entity', 'tfs', 'профессионально', 'новое', 'пример', 'оканчивать', 'просторный', 'кофемашина', 'печенка', 'книга', 'поощрение', 'ежегодный', 'стартапа'], ['знать', 'писать', 'читаемый', 'код', 'легко', 'следовать', 'гайд', 'практик', 'код', 'успех', 'нужно', 'знать', 'взаимодействовать', 'cocoa', 'design', 'patterns', 'and', 'design', 'autolayout', 'фрейм', 'autolayout', 'кроме', 'пара', 'радикальный', 'storyboard', 'стараться', 'убирать', 'interface', 'builder', 'код', 'это', 'нужно', 'знать', 'последний', 'тренд', 'swift', 'objective', 'c', 'писать', 'фич', 'swift', 'objective', 'c', 'это', 'objective', 'c', 'должный', 'code', 'style', 'code', 'review', 'coredata', 'поэтому', 'coredata', 'крайне', 'вэб', 'сокет', 'юнит', 'согласный', 'а+', 'ходьба', 'новочеркасский'

In [51]:
import gensim
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

dictionary = Dictionary(new_sentences)
corpus = [dictionary.doc2bow(t) for t in new_sentences]
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=12, update_every=1, chunksize=10000, passes=1)

lda.print_topics(10)

[(5,
  '0.034*"безопасность" + 0.029*"информационный" + 0.017*"защита" + 0.011*"информация" + 0.010*"интернет" + 0.006*"сетевой" + 0.006*"иб" + 0.006*"средство" + 0.006*"бизнес" + 0.005*"управление"'),
 (2,
  '0.007*"настройка" + 0.005*"сетевой" + 0.004*"data" + 0.004*"сервер" + 0.004*"веб" + 0.003*"процесс" + 0.003*"установка" + 0.003*"обработка" + 0.003*"java" + 0.003*"алгоритм"'),
 (6,
  '0.009*"sql" + 0.006*"заказчик" + 0.005*"бизнес" + 0.005*"решение" + 0.004*"проект" + 0.004*"данные" + 0.004*"процесс" + 0.004*"web" + 0.004*"управление" + 0.004*"ита"'),
 (8,
  '0.012*"проект" + 0.012*"база" + 0.006*"данные" + 0.005*"рассылка" + 0.005*"контент" + 0.004*"формирование" + 0.004*"управление" + 0.004*"google" + 0.004*"модель" + 0.003*"рекламный"'),
 (10,
  '0.012*"бизнес" + 0.011*"процесс" + 0.010*"oracle" + 0.009*"защита" + 0.007*"средство" + 0.007*"ms" + 0.006*"документация" + 0.006*"информация" + 0.006*"администрирование" + 0.005*"безопасность"'),
 (9,
  '0.038*"тестирование" + 0.011